In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve 
from sklearn.metrics import confusion_matrix, f1_score, fbeta_score, confusion_matrix
from xgboost.sklearn import XGBClassifier


from collections import Counter

from helper import clean_churn_df, model_baseline, model_baseline_no_cv, score_model_no_cv, score_model
from helper import split_with_dupe_rows_in_train, rf_no_cv_iterx

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler

from sqlalchemy import create_engine

plt.style.use('ggplot')
%matplotlib inline


%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from sqlid import sql_id
engine = create_engine(sql_id()) #stored my unique SQL identifier in a variable, set 

churn_df = pd.read_sql_query('select * from hr',con=engine)  
churn_df.drop('index', axis=1,inplace=True)
X_train, X_val, X_holdout, y_train, y_val, y_holdout = split_with_dupe_rows_in_train(churn_df)

In [7]:
# add improvements from feature engineering
X_train['hours_eval'] = X_train['average_montly_hours'] * X_train['last_evaluation']
X_val['hours_eval'] = X_val['average_montly_hours'] * X_val['last_evaluation']

In [25]:
model_baseline_no_cv(X_train,y_train,X_val, y_val)

{'Logistic regression': 0.6294706723891274,
 'KNN': 0.8275862068965518,
 'Gaussian Naive Bayes': 0.4669201520912547,
 'Multinomial Naive Bayes': 0.4213197969543147,
 'Support Vector Classifier': 0.9023383768913343,
 'Random Forest': 0.9690140845070422}

In [26]:
rf = RandomForestClassifier(random_state=41)
rf.fit(X_train, y_train)

svm_model = svm.SVC()
svm_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=41, verbose=0, warm_start=False)

In [38]:
from sklearn.ensemble import VotingClassifier
#create a dictionary of our models
estimators=[('svm', svm_model), ('rf', rf)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='hard')

In [39]:
ensemble.fit(X_train, y_train)
recall_score(ensemble.predict(X_val), y_val)

0.9907407407407407

Combo got a better score!